In [2]:
%load_ext autoreload
%autoreload 2
import sys,os
import pickle
from eknot_utils import EventNode,NodeParams
from utils import loadPickle,triHits,norm2,printTopWords
from eknot_utils import *

[resDocInd,tweetsObj,tweetsObjDedup,tweetsScore]= loadPickle('../output/summary_20160107_0114_80_0.pickle')
[Xs,vects,DT,ind2obj] = loadPickle('../data/20160107_0114.pickle')
rootParams,rootNodeDescriptor = loadPickle('../output/plsa_20160107_0114_80.pickle')

from time import time

In [13]:
topK = 20 # ent used
i = 0
t0 = time()
Pw_zs = rootNodeDescriptor.Pw_zs
Pe_z = Pw_zs[1][:,i]
evocab = vects[5].get_feature_names()
vocab = vects[0].get_feature_names()

ent_ind,ents = getEntInd(evocab,Pe_z,topK)

print( "entscore in "+str(time() - t0))
t0 = time()

newsObj = [ind2obj[n] for n in resDocInd]
XN,XEn,NEb,sentencesIn,sentencesInObj = getNewsContext(newsObj,ent_ind,ents,vocab)
print( "get news Context in "+str(time() - t0))
print len(newsObj),len(sentencesIn),len(set(sentencesIn))
t0 = time()

XT,XEt,TEb,tweetsIn,tweetsInObj = getTweetContext(tweetsObjDedup,ent_ind,ents,vocab)
print( "get tweet Context in "+str(time() - t0))




entscore in 0.0453150272369
get news Context in 0.682107925415
18 294 292
append in 32.7112281322
vectorize in 25.2670509815
ents append + vec in 5.39676213264
get tweet Context in 63.390681982


In [14]:
_XEn = XEn
_XEt = XEt
_NEb = NEb
_TEb = TEb

In [15]:
"""
#####
rows = []
interestedEnts = ['tennessee_loc_|','7-eleven_org_|','odds_other_|']

for e in interestedEnts:
    rows.append(ent_ind[e])
rows = np.array(rows)
#rows = np.array([10,33,34,40])
XEn = _XEn[rows,:]
XEt = _XEt[rows,:]
NEb = _NEb[:,rows]
TEb = _TEb[:,rows]
#####
"""
print len(tweetsObjDedup),len(tweetsIn),len(set(tweetsIn))
t0 = time()

newsScore = XN.dot(Pw_zs[0][:,i])

#tweetsScore = XT.dot(Pw_zs[0][:,i])
tweetsScore = []
for t in tweetsInObj:
    tweetsScore.append(float(t.pop))
tweetsScore = np.array(tweetsScore)

print( "init score in "+str(time() - t0))
t0 = time()

NE_ = XN.dot(XEn.T) #.multiply(NEb)
TE_ = XT.dot(XEt.T) #.multiply(TEb)
NE,EN = normBypartite(NE_)
TE,ET = normBypartite(TE_)
print( "graph constr in "+str(time() - t0))
t0 = time()

nScore, tScore = triHits(newsScore, tweetsScore, NE, EN, TE, ET, 0.2, 0.2, 5)
print( "trihits in "+str(time() - t0))
t0 = time()

43813 42455 42455
init score in 0.0731830596924
graph constr in 0.146972179413
==n converge 2 7.90854238998e-08
trihits in 0.0223200321198


In [16]:
nScore, tScore = triHits(np.ones(newsScore.shape), np.ones(tweetsScore.shape), NE, EN, TE, ET, 1, 1, 50)
print np.linalg.norm(norm2(newsScore)[:10] - nScore[:10])
print "========="
kSummary=20
printSummary(norm2(newsScore),norm2(tweetsScore),sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,kSummary)
print "*****"
printSummary(nScore,tScore,sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,kSummary)

==n converge 2 3.29022496214e-08
0.0436067020906
216 0.134491649562 2016-01-07 02:17:45 No Winner in  Powerball Lottery,  Jackpot Now Estimated $675 Million | No Winner in Powerball Lottery, Jackpot Now Estimated $675 Million
122 0.126589386593 2016-01-12 08:59:00 Powerball  hits $1.5 billion, largest-ever lottery  jackpot for one winner | Powerball hits $1.5 billion, largest-ever lottery jackpot for one winner
109 0.124220649441 2016-01-13 22:01:01 California lottery says it has a winner in $1.6 billion  Powerball  jackpot | California lottery says it has a winner in $1.6 billion Powerball jackpot
21 0.109463485842 2016-01-07 10:11:15 (Matt Rourke/ AP) The  Powerball  lottery  jackpot has already jumped to $700 million. | Powerball jackpot, already at a record level, could reach $1 billion
270 0.107324769868 2016-01-12 10:05:48 When you win the $1.5 billion  Powerball  lottery, here's what you should do | When you win the $1.5 billion Powerball lottery, here's what you should do
187 0

In [17]:
print NE.shape
res1 = printRankedEnt(NE.T.dot(nScore),ents,100)
print "===="
res2 = printRankedEnt(TE.T.dot(tScore),ents,100)
print "==="
for e in ents:
    print res2[e] - res1[e],e

(294, 20)
====
===
0 powerball_other_|
0 progressive_jackpot_other_|
-11 lottery_other_|
-2 chino_hills,_california_loc_|
-8 california_state_lottery_other_|
5 7-eleven_org_|
2 prize_other_|
2 california_loc_|
-5 florida_loc_|
-1 tennessee_loc_|
10 the_tonight_show_other_|
-3 territories_of_the_united_states_other_|
-4 publix_org_|
1 illinois_state_lottery_other_|
2 brain_other_|
8 gambling_other_|
-8 multi-state_lottery_association_other_|
10 billionaire_other_|
4 financial_adviser_other_|
-2 odds_other_|


In [14]:
count = 0
for i in range(len(newsObj)):
    b_ent = newsObj[i].b_ent
    sentences = sent_detector.tokenize(b_ent.strip())
    
    count += len(sentences)
print count

for sentence in sentences:
    print sentence
    s = makeEntText(sentence,ent_text)    
    print "---"
    print s
    print "====="

591
United_States_loc_|Americans_| have a chance to join the  Billionaire_other_|billionaire_|'s club Wednesday night, with the  Powerball_other_|Powerball_| reaching a historic $1.5 billion.


NameError: name 'ent_text' is not defined

In [22]:
for i in range(topK):
#i=18
    print ents[i]
    printTopWords(i,XEn,vocab,10)
    printTopWords(i,XEt,vocab,10)
    print "+++++++++"

powerball_other_|
[u'powerball', u'jackpot', u'lottery', u'million', u'tickets', u'billion', u'drawing', u'officials', u'states', u'state']
[u'powerball', u'jackpot', u'win', u'billion', u'winning', u'ticket', u'buy', u'know', u'lottery', u'million']
+++++++++
progressive_jackpot_other_|
[u'jackpot', u'million', u'powerball', u'lottery', u'billion', u'officials', u'drawing', u'winner', u'winners', u'winning']
[u'jackpot', u'powerball', u'billion', u'$1.5', u'lottery', u'winning', u'million', u'win', u'winner', u'winners']
+++++++++
lottery_other_|
[u'lottery', u'jackpot', u'officials', u'powerball', u'tickets', u'state', u'million', u'billion', u'california', u'wednesday']
[u'lottery', u'ticket', u'powerball', u'win', u'buy', u'billion', u'winning', u'bought', u'jackpot', u'lotto']
+++++++++
chino_hills,_california_loc_|
[u'hills', u'chino', u'store', u'california', u'7', u'lottery', u'sold', u'wednesday', u'tickets', u'night']
[u'chino', u'hills', u'powerball', u'winning', u'ticket', 